In [ ]:
from datetime import datetime
from os import environ
from os.path import join

import requests
import json

input_folder = environ.get(
    'CROSSCOMPUTE_INPUT_FOLDER', 'batches/standard/input')
output_folder = environ.get(
    'CROSSCOMPUTE_OUTPUT_FOLDER', 'batches/standard/output')

input_path = join(input_folder, 'variables.dictionary')
output_path = join(output_folder, 'variables.dictionary')

value_by_key = json.load(open(input_path, 'rt'))

In [ ]:
# 
WINTER_MONTHS = {
    "NORTHERN": [12, 1, 2],
    "SOURTHERN": [6, 7, 8]
}

COLD_SEASON = {
    "NORTHERN": [10, 11, 12, 1, 2, 3],
    "SOURTHERN": [4, 5, 6, 7, 8, 9]
}
WARM_SEASON = {
    "NORTHERN": [4, 5, 6, 7, 8, 9],
    "SOURTHERN": [10, 11, 12, 1, 2, 3]
}

In [ ]:
def limit_value(value, left_limit, right_limit):
    if value < left_limit:
        return left_limit
    if value > right_limit:
        return right_limit
    return value

In [ ]:
BASE_URL = "https://archive-api.open-meteo.com/v1/archive"
PREVIOUS_YEAR = datetime.now().year - 1

LATITUDE = limit_value(value_by_key["latitude"], -90, 90)
LONGITUDE = limit_value(value_by_key["longitude"], -180, 180)

HEMISPHERE = "NORTHERN" if float(LATITUDE) >= 0  else "SOURTHERN"

PARAMETERS = {
  "latitude": LATITUDE,
  "longitude": LONGITUDE,
  "start_date": f"{PREVIOUS_YEAR}-01-01",
  "end_date": f"{PREVIOUS_YEAR}-12-31",
  "hourly": "temperature_2m,relativehumidity_2m,precipitation"  
}

parameters = "&".join([f"{key}={value}" for key, value in PARAMETERS.items()])
URL = f"{BASE_URL}?{parameters}"

In [ ]:
response = requests.get(URL)

# if response.status_code != 200:
data = response.json()

In [ ]:
import pandas as pd

regions_summary = {}

df = pd.DataFrame.from_dict(data['hourly'])
df

## Heating degree days (HDD) 

> Heating degree days (HDD) are a measure of how cold the temperature was on a given day or during a period of days. For example, a day with a mean temperature of 40°F has 25 HDD. Two such cold days in a row have a total of 50 HDD for the two-day period.

https://www.eia.gov/energyexplained/units-and-calculators/degree-days.php
https://www.investopedia.com/terms/h/heatingdegreeday.asp

In [ ]:
df["time"] =  pd.to_datetime(df["time"])
df["HDD"] = 18.3333 - df["temperature_2m"]
df.HDD[df["HDD"] < 0] = 0
df

In [ ]:
# Group data by months
months_summary = {}

for month_number in range(1, 13):
    month_data = df[df['time'].dt.month == month_number]
    total_hours = len(month_data)
    month_summary = {
        "total_hours": total_hours,
        "temperature": month_data["temperature_2m"].sum() / total_hours,
        "relativehumidity": month_data["relativehumidity_2m"].sum() / total_hours, # 
        "precipitation": month_data["precipitation"].sum() / 10, # manage precipitation in centimeters instead of milimeters
        "HDD": month_data["HDD"].sum() / 24

    }
    months_summary[month_number] = month_summary
    
months_summary

In [ ]:
## Calculate six warmest months

temperature = [month["temperature"] for month in months_summary.values()]

months = [0,1,2,3,4,5]
month_combinations = []
temperatures = []

def inc(months):
    return [month + 1 for month in months]

while months[0] != 7:
    month_combinations.append(months)
    total = sum([temperature[x] for x in months])
    temperatures.append(total)
    months = inc(months)
    

max_temp = 0
index = 0 
for x in range(0, len(temperatures)):
    if max_temp < temperatures[x]:
        max_temp = temperatures[x]
        index = x
        

temperatures
six_warmest_months = inc(month_combinations[index])

temperatures[index], six_warmest_months

In [ ]:
# Precipitation = (rain + showers + snow)
annual_precipitation = sum([month["precipitation"] for month in months_summary.values()])

annual_heating_hours = len(df[df['temperature_2m'] >= 18.3333])
heating_degree_days = sum([month["HDD"] for month in months_summary.values()])

annual_summary = {
    "precipitation": annual_precipitation,
    "six_warmest_months": six_warmest_months,
    "heating_hours": annual_heating_hours,
    "heating_degree_days": heating_degree_days
}

annual_summary

# Hot-Humid

A hot-humid climate is defined as a region that:

  - [X] Receives more than 20 inches (50 cm) of annual precipitation and where one or both of the following occur:
    1. A 67°F (19.5°C) or higher wet bulb temperature for 3,000 or more hours during the warmest six consecutive months of the year; or
    2. A 73°F (23°C) or higher wet bulb temperature for 1,500 or more hours during the warmest six consecutive months of the year.
    
The Building America hot-humid climate zone includes the portions of IECC zones 1, 2, and 3 that are in the moist category (A) below the “warm-humid” line shown on the IECC map. 


In [ ]:
# Hot-Humid calculations
month_data = df[df['time'].dt.month.isin(six_warmest_months)]

six_warmest_months_total_hours = len(month_data)
six_warmest_months_hours_gte_19 = len(month_data[month_data["temperature_2m"] > 19.5])
six_warmest_months_hours_gte_23 = len(month_data[month_data["temperature_2m"] > 23])

is_hot_humid = annual_summary["precipitation"] > 50 and (six_warmest_months_hours_gte_19 > 3000 or six_warmest_months_hours_gte_23 > 1500)

regions_summary["Hot-Humid"] = {
    "selected":  is_hot_humid,
    "Receives more than 20 inches (50 cm) of annual precipitation": annual_summary["precipitation"] > 50,
    "A 67°F (19.5°C) or higher wet bulb temperature for 3,000 or more hours during the warmest six consecutive months of the year": six_warmest_months_hours_gte_19 > 3000,
    "Or a 73°F (23°C) or higher wet bulb temperature for 1,500 or more hours during the warmest six consecutive months of the year": six_warmest_months_hours_gte_23 > 1500
}
regions_summary

# Mixed-Humid
A mixed-humid climate is defined as a region that:

  - [X] Receives more than 20 inches (50 cm) of annual precipitation, 
  - [X] Has approximately 5,400 heating degree days 65°F basis (18.3333 °C) or fewer, and 
  - [X] Where the average monthly outdoor temperature drops below 45°F (7°C) during the winter months.
  
The Building America mixed-humid climate zone includes the portions of IECC zones 4 and 3 in category A above the “warmhumid” line. 

In [ ]:
def hdd_f2C(hdd_days):
    return (hdd_days * 9) / 5

In [ ]:
## Mixed-Humid

winter_data = df[df['time'].dt.month.isin(WINTER_MONTHS[HEMISPHERE])]
winter_avg_temperature = winter_data['temperature_2m'].sum() / len(winter_data) 
winter_avg_drops_below_7_C = winter_avg_temperature < 7

winter_avg_temperature, winter_avg_drops_below_7_C , annual_summary["heating_degree_days"], hdd_f2C(5400)

In [ ]:
is_mixed_humid = annual_summary["precipitation"] > 50 and winter_avg_drops_below_7_C and annual_summary["heating_degree_days"] < hdd_f2C(5400)

regions_summary["Mixed-Humid"] = {
    "selected":  is_mixed_humid,
    "Receives more than 20 inches (50 cm) of annual precipitation": annual_summary["precipitation"] > 50,
    "Has approximately 5,400 heating degree days 65°F basis (18.3333 °C) or fewer": annual_summary["heating_degree_days"] < hdd_f2C(5400),
    "Where the average monthly outdoor temperature drops below 45°F (7°C) during the winter months": winter_avg_drops_below_7_C
}

is_mixed_humid

## Hot-Dry
A hot-dry climate is defined as:
  - [X] A region that receives less than 20 inches (50 cm) of annual precipitation and
  - [X] where the monthly average outdoor temperature remains above 45°F (7°C) throughout the year.

The Building America hot-dry climate zone corresponds to the
portions of IECC zones 2 and 3 in the dry category.

In [ ]:
montly_average_outdoor_temp = sum([month['temperature'] for month in months_summary.values() ]) / 12
montly_average_outdoor_temp

In [ ]:
is_hot_dry = annual_summary["precipitation"] < 50 and montly_average_outdoor_temp > 7

regions_summary["Hot-Dry"] = {
    "selected":  is_hot_dry,
    "A region that receives less than 20 inches (50 cm) of annual precipitation": annual_summary["precipitation"] < 50,
    "where the monthly average outdoor temperature remains above 45°F (7°C) throughout the year": montly_average_outdoor_temp > 7
}
is_hot_dry

# Mixed-Dry

A mixed-dry climate is defined as:

- [X] A region that receives less than 20 inches (50 cm) of annual precipitation, 
- [X] Has approximately 5,400 heating degree days (65°F basis) or less, and 
- [X] Where the average monthly outdoor temperature drops below 45°F (7°C) during the winter months.
    
The Building America mixed-dry climate zone corresponds to IECC climate zone 4 B (dry).

In [ ]:
# Mixed-Dry
is_mixed_dry = annual_summary["precipitation"] < 50 and annual_summary["heating_degree_days"] < hdd_f2C(5400) and winter_avg_drops_below_7_C
regions_summary["Mixed-Dry"] = {
    "selected":  is_mixed_dry,
    "A region that receives less than 20 inches (50 cm) of annual precipitation": annual_summary["precipitation"] < 50,
    "Has approximately 5,400 heating degree days (65°F basis) or less": annual_summary["heating_degree_days"] < hdd_f2C(5400),
    "where the monthly average outdoor temperature remains above 45°F (7°C) throughout the year": winter_avg_drops_below_7_C
}
is_mixed_dry

# Cold

A cold climate is defined as:

- [X] A region with between 5,400 and 9,000 heating degree days (65°F basis).
    
The Building America cold climate corresponds to the IECC climate zones 5 and 6.

https://www.degreedays.net/introduction#:~:text=Fahrenheit%2Dbased%20degree%20days%20are,temperature%20of%2059%C2%B0F.

In [ ]:
is_cold =  annual_summary["heating_degree_days"] >= hdd_f2C(5400) and annual_summary["heating_degree_days"] <= hdd_f2C(9000)

regions_summary["Cold"] = {
    "selected":  is_cold,
    "A region with between 5,400 and 9,000 heating degree days (65°F basis)": annual_summary["heating_degree_days"] >= hdd_f2C(5400) and annual_summary["heating_degree_days"] <= hdd_f2C(9000)
}

(is_cold, hdd_f2C(5400), annual_summary["heating_degree_days"], hdd_f2C(9000))

# Very-Cold
A very cold climate is defined as:

- [X] a region with between 9,000 and 12,600 heating degree days (65°F basis).

The Building America very cold climate corresponds to IECC climate zone 7.

In [ ]:
is_very_cold =  annual_summary["heating_degree_days"] >= hdd_f2C(9000) and  annual_summary["heating_degree_days"] <= hdd_f2C(12600)
regions_summary["Very-Cold"] = {
    "selected":  is_very_cold,
    "A region with between 9,000 and 12,600 heating degree days (65°F basis)": annual_summary["heating_degree_days"] >= hdd_f2C(9000) and  annual_summary["heating_degree_days"] <= hdd_f2C(12600),
}
(is_very_cold, hdd_f2C(9000), annual_summary["heating_degree_days"], hdd_f2C(12600))

# Subarctic

A subarctic climate is defined as:
- [X] A region with 12,600 heating degree days (65° basis) or more. 

The only subarctic regions in the United States are in found Alaska, which is not shown in Figure 1.
The Building America subarctic climate zone corresponds to IECC climate zone 8.

In [ ]:
is_subartic = annual_summary["heating_degree_days"] >= hdd_f2C(12600)
regions_summary["Subarctic"] = {
    "selected":  is_subartic,
    "A region with 12,600 heating degree days (65° basis) or more": annual_summary["heating_degree_days"] >= hdd_f2C(12600),
}
is_very_cold, hdd_f2C(12600), annual_summary["heating_degree_days"]

# Marine

A marine climate is defined as a region that meets all of the
following criteria:

- [X] A coldest month mean temperature between 27°F (-3°C) and 65°F (18°C)
- [X] A warmest month mean of less than 72°F (22°C)
- [X] At least 4 months with mean temperatures higher than 50°F (10°C)
- [ ] A dry season in summer. The month with the heaviest precipitation in the cold season has at least three times as much precipitation as the month with the least precipitation in the rest of the year. 
- [ ] The cold season is October through March in the Northern Hemisphere and April through September in the Southern Hemisphere.
      
The Building America marine climate corresponds to those portions of IECC climate zones 3 and 4 located in the “C” moisture category. 

In [ ]:
coldest_month = min([month['temperature'] for month in months_summary.values()])
coldest_month_in_range = coldest_month >= -3 and coldest_month <= 18

warmest_month = max([month['temperature'] for month in months_summary.values()])
warmest_month_in_range = coldest_month < 22

mimimum_warmest_months = len([month['temperature'] for month in months_summary.values() if month['temperature'] > 10])

month_heaviest_precipitation = max([month['precipitation'] for month in map(months_summary.get, COLD_SEASON[HEMISPHERE])])
month_least_precipitation = min([month['precipitation'] for month in map(months_summary.get, WARM_SEASON[HEMISPHERE])])

dry_season_in_summer = month_heaviest_precipitation / month_least_precipitation > 3

coldest_month, coldest_month_in_range, warmest_month, warmest_month_in_range, mimimum_warmest_months, dry_season_in_summer

In [ ]:
is_marine = coldest_month_in_range and warmest_month_in_range and mimimum_warmest_months >=4  and dry_season_in_summer
regions_summary["Marine"] = {
    "selected":  is_marine,
    "A coldest month mean temperature between 27°F (-3°C) and 65°F (18°C)": coldest_month_in_range,
    "A warmest month mean of less than 72°F (22°C)": warmest_month_in_range,
    "At least 4 months with mean temperatures higher than 50°F (10°C)": mimimum_warmest_months >=4,
    "A dry season in summer. The month with the heaviest precipitation in the cold season has at least three times as much precipitation as the month with the least precipitation in the rest of the year.": dry_season_in_summer    
}
is_marine

In [ ]:
value_by_key = {
  "climate_region": ", ".join([key for key, value in regions_summary.items() if value["selected"]]),
  "summary": regions_summary
}

import numpy as np

class CustomJSONizer(json.JSONEncoder):
    def default(self, obj):
        return super().encode(bool(obj))  if isinstance(obj, np.bool_) else super().default(obj)

json.dump(value_by_key, open(output_path, 'wt'), indent=4, cls=CustomJSONizer)
value_by_key